In [4]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document
import os

In [5]:
documents = [
    Document(page_content='"Regular physical activity improves overall health and reduces the risk of chronic diseases.', metadata={'source':'H1'}),
    Document(page_content='A balanced diet rich in fruits, vegetables, and whole grains supports optimal wellness.', metadata={'source':'H2'}),
    Document(page_content='Meditation can reduce stress and improve mental clarity and emotional well-being', metadata={'source':'H3'}),
    Document(page_content='Quality sleep is essential for physical restoration and cognitive function', metadata={'source':'H4'}),
    Document(page_content='Drinking enough water is fundamental for maintaining bodily functions and energy levels.', metadata={'source':'H4'}),
    Document(page_content='Lifestyle changes and natural remedies can help manage anxiety symptoms effectively.', metadata={'source':'H5'}),
    Document(page_content='Yoga enhances flexibility, strength, and mental relaxation.', metadata={'source':'I1'}),
    Document(page_content='Heart-healthy habits can significantly lower the risk of cardiovascular diseases.', metadata={'source':'I2'}),
    Document(page_content='Essential nutrients support the immune system and overall health maintenance.', metadata={'source':'I3'}),
    Document(page_content='Effective strategies can help reduce stress and promote emotional resilience.', metadata={'source':'I4'}),
    Document(page_content='Effective strategies can help reduce stress and promote emotional resilience.', metadata={'source':'I5'}),
]

In [6]:
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"
os.environ["OPENAI_API_KEY"] = "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"

model = ChatOpenAI(model="deepseek/deepseek-r1:free")

In [7]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"trust_remote_code": True}
)

vector_store = FAISS.from_documents(
    documents=documents,
    embedding = embeddings
)

C:\Users\gauta\AppData\Local\Temp\ipykernel_10984\3775599942.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [9]:
base_retriever = vector_store.as_retriever(search_kwargs={'k':5})
compressor = LLMChainExtractor.from_llm(model)

In [11]:
compressor_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [12]:
query = 'what is balanced diet'
compressed_result = compressor_retriever.invoke(query)

In [13]:
for i, doc in enumerate(compressed_result):
    print(f'\n----------Result {i+1}--------')
    print(f'Content:\n{doc.page_content}')


----------Result 1--------
Content:
Extracted relevant parts: A balanced diet rich in fruits, vegetables, and whole grains supports optimal wellness.

----------Result 2--------
Content:
Extracted relevant parts: Essential nutrients support the immune system and overall health maintenance.
